In [3]:
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:
from typing import List, Optional
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain.chat_models import init_chat_model

# Initialise LLM (Gemini 2.5)
llm = init_chat_model(
    model="gemini-2.5-flash",
    model_provider="google_genai",
    temperature=0.0
)

# Canonical user input
paragraph = (
    "I identified these five genes to be significantly more mutated than expected by chance in my cohort of human brain cancer patients: TP53, AKT3, EGFR, ATRX and PDX1."
)

# Define Pydantic class for input genes and context
class StudyExtraction(BaseModel):
    genes: List[str] = Field(
        description="List of gene symbols mentioned in the text, normalized to official HGNC/NCBI-style symbols if possible."
    )
    organism: Optional[str] = Field(
        description="Scientific name (binomial) of the organism (e.g., 'Homo sapiens', 'Mus musculus')."
    )
    field_of_study: Optional[str] = Field(
        description="High-level biomedical domain, e.g., 'oncology', 'cancer genomics', 'neuroscience', 'immunology', 'microbiology'."
    )
    organ: Optional[str] = Field(
        description="Primary organ or tissue referenced (e.g., 'brain', 'liver', 'blood')."
    )
    analysis_type: Optional[str] = Field(
        description="Concise description of the analysis performed, e.g., 'differential expression', 'mutation enrichment', 'GWAS', 'copy-number analysis', 'metagenomic profiling'."
    )
    GSEA: bool = Field(default=False, 
                       description="Whether the user mentions that a GSEA is needed on the gene set. If no mention, keep it False.")

#alternative implementation to parse as pydantic more robustly
parser = PydanticOutputParser(pydantic_object=StudyExtraction)
format_instructions = parser.get_format_instructions()

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract per schema:\n{format_instructions}"),
    ("human", "{paragraph}"),
]).partial(format_instructions=format_instructions)
parsing_llm = prompt | llm | parser

# pass raw user input "paragraph"
parsed_input = parsing_llm.invoke({"paragraph": paragraph})
#convert to JSON string
json_output = parsed_input.model_dump_json(indent=2)
print(json_output)

#next step - inject the json to the LLM to determine attributes to fetch from BioMart


{
  "genes": [
    "TP53",
    "AKT3",
    "EGFR",
    "ATRX",
    "PDX1"
  ],
  "organism": "Homo sapiens",
  "field_of_study": "oncology",
  "organ": "brain",
  "analysis_type": "mutation enrichment"
}


In [6]:
from src.querries_script import group_by_gene_dynamic, fill_with_ncbi, call_querry_biomart
import pandas as pd    

attributes = pd.read_csv("data/attributes.csv")["name"].to_list()

output = call_querry_biomart(attributes=attributes[:15],
                            filters={"external_gene_name": ["TLR4", "NOTCH1"]})

output = group_by_gene_dynamic(output)

output = fill_with_ncbi(output)

print(output)

[{'ensembl_gene_id': 'ENSG00000148400', 'description': 'notch receptor 1 [Source:HGNC Symbol;Acc:HGNC:7881]', 'chromosome_name': '9', 'start_position': '136494433', 'end_position': '136546048', 'strand': '-1', 'band': 'q34.3', 'external_gene_name': 'NOTCH1', 'transcript_count': '16', 'percentage_gene_gc_content': '63.46', 'gene_biotype': 'protein_coding', 'external_synonym': 'TAN1', 'phenotype_description': 'Adams-Oliver syndrome', 'name_1006': ['Notch signaling pathway', 'aortic valve morphogenesis', 'cellular response to follicle-stimulating hormone stimulus', 'cellular response to tumor cell', 'cellular response to vascular endothelial growth factor stimulus'], 'namespace_1003': 'biological_process', 'go_id': ['GO:0007219', 'GO:0003180', 'GO:0071372', 'GO:0071228', 'GO:0035924'], 'go_linkage_type': ['IDA', 'IMP'], 'ncbi': 'Gene: NOTCH1 (Homo sapiens)\n\n--- Summary ---\nThis gene encodes a member of the NOTCH family of proteins. Members of this Type I transmembrane protein family sh